# Physical Space and the Image Grid

### Learning Objectives

* Understand how an **image pixel grid** is defined in **physical space**
* Understand how a **spatial transform** *defines the map between two physical spaces*
* Understand **how to define a resampling operation** to generate a new image grid
* Understand the **role of image interpolators**, and become familiar with a few common interpolators

## Overview

### Physical Space, Images, and the Pixel Grid

When performing image registration, where the object is to find the spatial transform that maps between the fixed image space and the moving image space, it is **extremely important** to have clear definitions of **physical space**, aka **world space**, and how images live in these spaces.

Once we have *good definitions* and *follow them consistently*, operations such resampling, interpolation, multi-resolution processing, and spatial transform as are **clear** and **correct**, even on images that **anisotropic**, **oriented**, or with **multi-component tensor pixels**, such as gradient vectors or diffusion tensors.

### Problem: Register a Human Lung Volume from Two Time Points

Given a problem to register x-ray computed tomography (CT) images of a human lung taken at two time points, **what must be supported?**

In image registration, we typically identify the two images as the **fixed** and **moving** image. Our goal is to **find the spatial transformation that makes the moving image so align with the fixed image**. Our fixed image was acquired from the patient at a baseline time point, and our moving image was acquired at a follow-up time point.

First, let's examine our fixed and moving images.

We will use the [`itkwidgets`](https://github.com/InsightSoftwareConsortium/itkwidgets), interative Jupyter widgets to visualize images, point sets, and meshes in 2D and 3D.

The `itkwidgets` are built on [The Insight Toolkit (ITK)](https://github.com/InsightSoftwareConsortium/ITK). ITK is an open-source, cross-platform toolkit for N-dimensional scientific image processing, segmentation, and registration. ITK has been collaboratively developed by a large community of research software engineers for over 20 years and serves as the foundation for many popular image registration tools such as [ANTs](https://github.com/ANTsX/ANTsPy), [elastix](http://elastix.isi.uu.nl/), or [ndreg](https://github.com/InsightSoftwareConsortium/ITKNDReg/).

ITK is one of the few libraries that supports a wide variety of parametric spatial transformation models and an HDF5 file format to serialize and deserialize these models. Spatial transformation models range from rigid, similarity, affine, kernel-based spline, b-spline, to dense displacement fields and composite transformations. The HDF5 file format, important for reproducibility, transform quantification, and avoiding resampling issues related to data size, precision, and interpolation, has been adopted by projects such as [3D Slicer](https://slicer.org/) for medical imaging, [DREAM.3D](dream3d.bluequartz.net/) for material science imaging, and the [Brain Imaging Data Structure (BIDS)](https://bids.neuroimaging.io/) for neuroimaging.

In [1]:
import itk
from itkwidgets import view, checkerboard

In [2]:
fixed_image = itk.imread('imgs/CT_3D_lung_fixed.mha')

view(fixed_image, mode='x')

Viewer(geometries=[], gradient_opacity=0.22, mode='x', point_sets=[], rendered_image=<itkImagePython.itkImageS…

The CT image provides radiodensity in [Hounsefield units](https://en.wikipedia.org/wiki/Hounsfield_scale); lung tissue approaches -1000 in intensity, the value for air, and soft tissue is slightly above 0 intensity, the value for water.

In [4]:
view(fixed_image, vmax=-300, gradient_opacity=0.5, ui_collapsed=True, annotations=False)

Viewer(annotations=False, geometries=[], gradient_opacity=0.5, point_sets=[], rendered_image=<itkImagePython.i…

In [5]:
view(fixed_image, vmin=-50, vmax=500, ui_collapsed=True, annotations=False)

Viewer(annotations=False, geometries=[], gradient_opacity=0.22, point_sets=[], rendered_image=<itkImagePython.…

In [6]:
print(fixed_image.GetSpacing())

view(fixed_image, mode='z')

itkVectorD3 ([1.366, 1.366, 2.5])


Viewer(geometries=[], gradient_opacity=0.22, mode='z', point_sets=[], rendered_image=<itkImagePython.itkImageS…

**Take-home observations**:

- The resolution is anisotropic; resolution is higher in-plane versus out-of-plane.
- The pixel type is signed short: registration requires supporting this pixel type and interpolating samples between these pixels.

We can compare the fixed and moving images by interleaving them in a checkboard pattern.

In [3]:
moving_image = itk.imread('imgs/CT_3D_lung_moving.mha')

checkerboard(moving_image, fixed_image,
             mode='v', ui_collapsed=False, slicing_planes=True, gradient_opacity=0.5, shading=False)

**Discussion question**: how is a checkerboard created if the images are not sampled on the same image grid?

We **conceptualize** a **spatial image** as a **uniform grid of pixels** or **pixel grid** that lives is physical space.

Let's continue [with this presentation](https://docs.google.com/presentation/d/1TGjGseDGa_IjrBWA5aMLzxKssy0Al5KT-_FmGlYF92Y/edit?usp=sharing).

### Exercise: Lung image size

There are two 3D CT lung images, sampled on different image grids:

1. *imgs/CT_3D_lung_fixed.mha*
2. *imgs/CT_3D_lung_fixed_small.mha*

Which is larger in terms of its:

* File size?
* Spatial domain?
* Resolution?

How much less time will it take to register *CT_3D_lung_fixed_small.mha* to *CT_3D_lung_moving.mha*?

In [1]:
!ls imgs

CT_3D_lung_fixed.mha	    CT_3D_lung_moving.mha  phantom.tif
CT_3D_lung_fixed_small.mha  phantom2.tif
